---
# Compute the Dependent Variable 'aggressiveLanguage'


---

OUTPUT FILE:
1. 'data_result_fifteen_subreddits_aggressive_language.csv': Contains the threat, insult, and toxicity scores for each comment

references for detoxify-<br>
https://www.kaggle.com/code/renokan/toxic-comments-using-detoxify-model/notebook?scriptVersionId=87256021 <br>
https://github.com/unitaryai/detoxify <br>

.



.

---
# **Part 1: Reading the data**

In this section, I have read the the output file 'data_fifteen_subreddits_similarity.csv' from the python notebook 'Comment_Level_Network_Cultural_Similarity.ipynb'. This file contains all the comments for which both network similarity and cultural similarity were computed.

---
.

Check if cuda is being used

In [1]:
import torch
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


Connect to drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


reading the file which contains the comments for which both network and cultural similarity, along with parent author was computed

In [27]:
import pandas as pd
import numpy as np
data_fifteen_subreddits_similarity = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/data_fifteen_subreddits_similarity.csv', low_memory=False, index_col = 0)
print(len(data_fifteen_subreddits_similarity)) #length of data = 107352
print(len(pd.unique(data_fifteen_subreddits_similarity['subreddit_id']))) #number of subreddits considered = 15
print(len(pd.unique(data_fifteen_subreddits_similarity['id']))) #unique number of comments = , the data is at the comment level =
print(len(pd.unique(data_fifteen_subreddits_similarity['parent_id']))) #number of parent nodes =
print(len(pd.unique(data_fifteen_subreddits_similarity['link_id']))) #number of submissions =
print(len(pd.unique(data_fifteen_subreddits_similarity['author']))) #number of submissions =
print(len(data_fifteen_subreddits_similarity.columns))
print(pd.unique(data_fifteen_subreddits_similarity['subreddit_id'])) #there are fifteen unique subreddits, the nana rows are ignored

58874
15
58874
38858
3017
23718
21
['t5_2qo4s' 't5_22i0' 't5_2qh33' 't5_2qhwp' 't5_2r4oc' 't5_2sgoq'
 't5_2scss' 't5_2vbli' 't5_2sjgc' 't5_2ror6' 't5_2r2jt' 't5_2qm35'
 't5_2wm0g' 't5_3deqz' 't5_2qmpb']


In [4]:
print(data_fifteen_subreddits_similarity.columns)

Index(['Unnamed: 0', 'edited', 'id', 'parent_id', 'distinguished',
       'created_utc', 'author_flair_text', 'author_flair_css_class',
       'controversiality', 'subreddit_id', 'retrieved_on', 'link_id', 'author',
       'score', 'gilded', 'stickied', 'body', 'subreddit',
       'network_similarity', 'cultural_similarity', 'parent_comment_author'],
      dtype='object')


In [5]:
data_fifteen_subreddits_similarity.head(3)

,Unnamed: 0,edited,id,parent_id,distinguished,created_utc,author_flair_text,author_flair_css_class,controversiality,subreddit_id,...,link_id,author,score,gilded,stickied,body,subreddit,network_similarity,cultural_similarity,parent_comment_author
64,64,0,dbumpbm,t1_dbumnvd,NaN,1.483229e+09,Wizards,Wizards3,0.0,t5_2qo4s,...,t3_5lbr78,Exoguana,13.0,0.0,False,I don't like to watch the Wizards. Gets me str...,nba,0.752598,0.514737,passiveparrot
70,70,0,dbumphm,t1_dbumo04,NaN,1.483229e+09,Kings,Kings1,0.0,t5_2qo4s,...,t3_5lbnxn,Stankie,1.0,0.0,False,You son of a bitch. Take it back. Take it all ...,nba,0.812986,0.147589,yungtito
96,96,0,dbumqp3,t1_dbumnzf,NaN,1.483229e+09,Er ist bisschen ein Otto geworden.,SHOL,0.0,t5_22i0,...,t3_5lbr04,SpaceHippoDE,1.0,0.0,False,achso das,de,0.780005,0.369992,oldandgreat


drop teh unnamed column

In [9]:
data_fifteen_subreddits_similarity = data_fifteen_subreddits_similarity.drop('Unnamed: 0', axis=1)

printy dataframe once again to check

In [10]:
data_fifteen_subreddits_similarity.head(3)

,edited,id,parent_id,distinguished,created_utc,author_flair_text,author_flair_css_class,controversiality,subreddit_id,retrieved_on,link_id,author,score,gilded,stickied,body,subreddit,network_similarity,cultural_similarity,parent_comment_author
64,0,dbumpbm,t1_dbumnvd,NaN,1.483229e+09,Wizards,Wizards3,0.0,t5_2qo4s,1.485680e+09,t3_5lbr78,Exoguana,13.0,0.0,False,I don't like to watch the Wizards. Gets me str...,nba,0.752598,0.514737,passiveparrot
70,0,dbumphm,t1_dbumo04,NaN,1.483229e+09,Kings,Kings1,0.0,t5_2qo4s,1.485680e+09,t3_5lbnxn,Stankie,1.0,0.0,False,You son of a bitch. Take it back. Take it all ...,nba,0.812986,0.147589,yungtito
96,0,dbumqp3,t1_dbumnzf,NaN,1.483229e+09,Er ist bisschen ein Otto geworden.,SHOL,0.0,t5_22i0,1.485680e+09,t3_5lbr04,SpaceHippoDE,1.0,0.0,False,achso das,de,0.780005,0.369992,oldandgreat


confirm that the following columns do not have missing values

In [11]:
print(data_fifteen_subreddits_similarity['network_similarity'].isna().sum())
print(data_fifteen_subreddits_similarity['cultural_similarity'].isna().sum())
print(data_fifteen_subreddits_similarity['parent_comment_author'].isna().sum())
print(data_fifteen_subreddits_similarity['body'].isna().sum())

0
0
0
0


check the user-parent value counts

In [12]:
print(len(data_fifteen_subreddits_similarity[['author', 'parent_comment_author']].value_counts()))

49972


that means there are 58874-49972 =  8902 repeated pairs of user-parent

---
# **Part 2: Compute the 'aggressiveLanguage' variables**


---
.

In [13]:
pip install detoxify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.5 MB/s eta 0:00:00


In [14]:
from detoxify import Detoxify

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [15]:
detoxify_model = Detoxify(
    model_type='original',
    device='cuda'
)

Downloading: "https://github.com/unitaryai/detoxify/releases/download/v0.1-alpha/toxic_original-c1212f89.ckpt" to /root/.cache/torch/hub/checkpoints/toxic_original-c1212f89.ckpt
100%|██████████| 418M/418M [00:01<00:00, 287MB/s]


(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [16]:
predicts_dict = detoxify_model.predict("sample sentence")
print(len(predicts_dict))
print(type(predicts_dict))
print(predicts_dict)
print(predicts_dict['threat'])

6
<class 'dict'>
{'toxicity': 0.0006331979, 'severe_toxicity': 0.00012220243, 'obscene': 0.0001922156, 'threat': 0.000112057016, 'insult': 0.00018101618, 'identity_attack': 0.00014015858}
0.000112057016


In [17]:
print(len(data_fifteen_subreddits_similarity))
data_fifteen_subreddits_similarity.head(3)

58874


,edited,id,parent_id,distinguished,created_utc,author_flair_text,author_flair_css_class,controversiality,subreddit_id,retrieved_on,link_id,author,score,gilded,stickied,body,subreddit,network_similarity,cultural_similarity,parent_comment_author
64,0,dbumpbm,t1_dbumnvd,NaN,1.483229e+09,Wizards,Wizards3,0.0,t5_2qo4s,1.485680e+09,t3_5lbr78,Exoguana,13.0,0.0,False,I don't like to watch the Wizards. Gets me str...,nba,0.752598,0.514737,passiveparrot
70,0,dbumphm,t1_dbumo04,NaN,1.483229e+09,Kings,Kings1,0.0,t5_2qo4s,1.485680e+09,t3_5lbnxn,Stankie,1.0,0.0,False,You son of a bitch. Take it back. Take it all ...,nba,0.812986,0.147589,yungtito
96,0,dbumqp3,t1_dbumnzf,NaN,1.483229e+09,Er ist bisschen ein Otto geworden.,SHOL,0.0,t5_22i0,1.485680e+09,t3_5lbr04,SpaceHippoDE,1.0,0.0,False,achso das,de,0.780005,0.369992,oldandgreat


check that no row has a missing body

In [18]:
data_fifteen_subreddits_similarity['body'].isnull().sum()

0

In [19]:
def aggressiveLanguage_mapper(input_data):

  input_data['insult_prob'] = np.nan
  input_data['toxicity_prob'] = np.nan
  input_data['threat_prob'] = np.nan

  ignore_comments_counter = 0
  j = 0

  #additional code to resolve an error
  type_base = type(input_data['parent_id'].iloc[0])

  for ind, row in input_data.iterrows():
    j += 1
    if j % 1000 == 0:
      print('finished comment '+str(j)+'/'+str(len(input_data)))

    curr_author = row['author']
    curr_subreddit_id = row['subreddit_id']
    curr_id = row['id']
    curr_comment_body = row['body']

    #check the aggressive language probability in the comment body
    predicts_dict = detoxify_model.predict(curr_comment_body)
    input_data.at[ind,'insult_prob'] = predicts_dict["insult"]
    input_data.at[ind,'toxicity_prob'] = predicts_dict["toxicity"]
    input_data.at[ind,'threat_prob'] = predicts_dict["threat"]

  print('total number of comments ignored: ' +str(ignore_comments_counter))
  return input_data


In [20]:
data_result_fifteen_subreddits = aggressiveLanguage_mapper(data_fifteen_subreddits_similarity)
print(len(data_result_fifteen_subreddits)) #length of data
print(len(pd.unique(data_result_fifteen_subreddits['subreddit_id']))) #number of subreddits considered =
print(len(pd.unique(data_result_fifteen_subreddits['id']))) #unique number of comments = , the data is at the comment level =
print(len(pd.unique(data_result_fifteen_subreddits['parent_id']))) #number of parent nodes =
print(len(pd.unique(data_result_fifteen_subreddits['link_id']))) #number of submissions =
print(len(data_result_fifteen_subreddits.columns))

finished comment 1000/58874
finished comment 2000/58874
finished comment 3000/58874
finished comment 4000/58874
finished comment 5000/58874
finished comment 6000/58874
finished comment 7000/58874
finished comment 8000/58874
finished comment 9000/58874
finished comment 10000/58874
finished comment 11000/58874
finished comment 12000/58874
finished comment 13000/58874
finished comment 14000/58874
finished comment 15000/58874
finished comment 16000/58874
finished comment 17000/58874
finished comment 18000/58874
finished comment 19000/58874
finished comment 20000/58874
finished comment 21000/58874
finished comment 22000/58874
finished comment 23000/58874
finished comment 24000/58874
finished comment 25000/58874
finished comment 26000/58874
finished comment 27000/58874
finished comment 28000/58874
finished comment 29000/58874
finished comment 30000/58874
finished comment 31000/58874
finished comment 32000/58874
finished comment 33000/58874
finished comment 34000/58874
finished comment 35000/

In [21]:
data_result_fifteen_subreddits.head(3)

,edited,id,parent_id,distinguished,created_utc,author_flair_text,author_flair_css_class,controversiality,subreddit_id,retrieved_on,...,gilded,stickied,body,subreddit,network_similarity,cultural_similarity,parent_comment_author,insult_prob,toxicity_prob,threat_prob
64,0,dbumpbm,t1_dbumnvd,NaN,1.483229e+09,Wizards,Wizards3,0.0,t5_2qo4s,1.485680e+09,...,0.0,False,I don't like to watch the Wizards. Gets me str...,nba,0.752598,0.514737,passiveparrot,0.082428,0.938899,0.001552
70,0,dbumphm,t1_dbumo04,NaN,1.483229e+09,Kings,Kings1,0.0,t5_2qo4s,1.485680e+09,...,0.0,False,You son of a bitch. Take it back. Take it all ...,nba,0.812986,0.147589,yungtito,0.958220,0.996265,0.001973
96,0,dbumqp3,t1_dbumnzf,NaN,1.483229e+09,Er ist bisschen ein Otto geworden.,SHOL,0.0,t5_22i0,1.485680e+09,...,0.0,False,achso das,de,0.780005,0.369992,oldandgreat,0.000281,0.003264,0.000097


confirm that there are no missing 'insult', 'threat', and 'toxicity' columns

In [23]:
print(data_result_fifteen_subreddits['insult_prob'].isnull().sum())
print(data_result_fifteen_subreddits['threat_prob'].isnull().sum())
print(data_result_fifteen_subreddits['toxicity_prob'].isnull().sum())

0
0
0


In [24]:
data_result_fifteen_subreddits.to_csv('/content/gdrive/MyDrive/Colab Notebooks/reddit_project/data_result_fifteen_subreddits_aggressive_language.csv')

In [25]:
print(len(data_result_fifteen_subreddits[['author', 'parent_comment_author']].value_counts()))

49972


that means there are 58874-49972 =  8902 repeated pairs of user-parent

---
Creating a Dummy Variable for Cultural Fit based on median Value

---

.

.



In [28]:
data_result_fifteen_subreddits["cultural_similarity"].median()

0.36710138618946075